In [1]:
# TODO DELETE ME
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [3]:
import pandas as pd
import math

# to set random seed
import numpy as np

# used to create CRF model
from sklearn_crfsuite import CRF

# used to evaluate model
from sklearn_crfsuite import metrics

# for specifying f1 metrics
from sklearn.metrics import make_scorer

# for cross validation of hyperparameters
from sklearn.model_selection import RandomizedSearchCV

# to visualize the weight of parameters of the fitted model
import eli5

<IPython.core.display.Javascript object>

In [4]:
pd.set_option("max_row", 600)

<IPython.core.display.Javascript object>

In [5]:
np.random.seed(42)

<IPython.core.display.Javascript object>

In [6]:
data = pd.read_csv("tagged_data.csv", index_col=0)

<IPython.core.display.Javascript object>

In [7]:
data.head(10)

,sentence#,word,pos,tag
0,0.0,Preheat,VB,U-Action
1,0.0,oven,NN,U-Utensil
2,0.0,to,IN,O
3,0.0,425,CD,O
4,0.0,degrees,NNS,O
5,0.0,F.,NN,O
6,1.0,Press,NN,U-Action
7,1.0,dough,NN,O
8,1.0,into,IN,O
9,1.0,the,DT,O


<IPython.core.display.Javascript object>

In [8]:
# Set the sentence# as index for merging ID:212 to ID:263
parse_data.set_index("sentence#", inplace=True)
parse_data.tail()

NameError: name 'parse_data' is not defined

<IPython.core.display.Javascript object>

In [ ]:
# Merge the words with same sentence#
combined_sentence = (
    parse_data.groupby(["sentence#"])["word"]
    .apply(lambda x: " ".join(x.astype(str)))
    .reset_index()
)
combined_sentence.head()

In [ ]:
# Convert the dataframe to the nested list
combined_sentences_list = combined_sentence.reset_index()[
    ["sentence#", "word"]
].values.tolist()
for item in combined_sentences_list:
    print(item[1])

In [ ]:
# print(combined_sentence_list)
dp_picked_sent = []
not_dp_picked_sent  = []

# Get the sentence for SpaCy
for sentence in combined_sentences_list:
    # Locate the sentence element
    ind_sentences = sentence[1]
    # Create SpaCy doc
    doc = nlp(ind_sentences)

    # Get the action words
    for token in doc:
        # print(token)

        # Pick all the Verb in pos_, and anything but advcl in dep_
        if token.pos_ == "VERB" and token.dep_ != "advcl":
            # print(int(sentence[0]), " ".join([token.dep_ for token in doc]))
            dp_parser = " ".join([token.dep_ for token in doc])
            dp_picked_sent.append((int(sentence[0]), dep_parser))
        ###################################################
        # HELP: How to pick up the rest of missed setence ID and setnece itself
        # else:
        #      not_dp_picked_sent.append(sentence[0])
        #####################################################
        # if token.dep_ == "ROOT" or token.dep_ == "conj" or token.dep_ == "nmod" or token.dep_ == "advcl":
        # print(doc, "---->>>")

In [ ]:
# Summary
print(
    "Total # of sentences:",
    len(combined_sentences_list),
    "\n# of Dependency Parser PICKED sentences:",
    len(set(dp_picked_sent)),
    "\nMISSED sentences ID:", 212, 213, 217, 226, 233, 259
)

In [ ]:
doc1 = nlp(
    "Remove from freezer and place waxed paper between each burger as you stack them ."
)
displacy.serve(doc1, style="dep")

In [ ]:
##########################
# The end
##########################

In [ ]:
data["tag"].value_counts()

In [ ]:
print(len(data["word"].values))

In [ ]:
words = list(set(data["word"].values))
len(words)

In [ ]:
agg_func = lambda s: [
    (w, p, t)
    for w, p, t in zip(
        s["word"].values.tolist(), s["pos"].values.tolist(), s["tag"].values.tolist()
    )
]

In [ ]:
grouped = data.groupby("sentence#").apply(agg_func)

In [ ]:
sentences = [s for s in grouped]

In [ ]:
len(sentences)

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        # capture the proportion of a given label in the training set
        "bias": 1.0,
        # get the lower case form of the word
        "word.lower()": word.lower(),
        # get last 3 letters for the word
        "word[-3:]": word[-3:],
        # get last 2 letters for the word
        "word[-2:]": word[-2:],
        # check whether the word is uppercase or not
        "word.isupper()": word.isupper(),
        # check whether the word is title case or not
        "word.istitle()": word.istitle(),
        # check whether the word is digit or not, useful to identifying quantities which will be tagged as 'O'
        "word.isdigit()": word.isdigit(),
        # specifying the pos for word
        "postag": postag,
        # get first 2 letters for the POS tag
        "postag[:2]": postag[:2],
        # check if word is symbol or not
        "non_symbol_checker": word not in ["(", ")", ".", ","],
    }

    # if word is starting of sentence
    if i > 0:

        # if word is not the beginning of sentence
        # then get the word before it i.e. i-1 index
        word1 = sent[i - 1][0]

        # then get the pos before it i.e. i-1 index
        postag1 = sent[i - 1][1]

        features.update(
            {
                # setting the lower form of word at index i-1
                "-1:word.lower()": word1.lower(),
                # checking if the word at index i-1 is titlecase
                "-1:word.istitle()": word1.istitle(),
                # checking if the word at index i-1 is uppercase
                "-1:word.isupper()": word1.isupper(),
                # setting the pos of word at index i-1
                "-1:postag": postag1,
                # get first 2 letters for the POS tag for i-1 indexed word
                "-1:postag[:2]": postag1[:2],
            }
        )
    else:
        # setting the BOS or Begining of sentence to True
        features["BOS"] = True

    # if word is at the end of sentence
    if i < len(sent) - 1:

        # if word is not the end of sentence
        # then get the word after it i.e. i+1 index
        word1 = sent[i + 1][0]

        # then get the pos after it i.e. i+1 index
        postag1 = sent[i + 1][1]

        features.update(
            {
                # setting the lower form of word at index i+1
                "+1:word.lower()": word1.lower(),
                # checking if the word at index i+1 is titlecase
                "+1:word.istitle()": word1.istitle(),
                # checking if the word at index i+1 is titlecase
                "+1:word.isupper()": word1.isupper(),
                # setting the pos of word at index i+1
                "+1:postag": postag1,
                # get first 2 letters for the POS tag for i+1 indexed word
                "+1:postag[:2]": postag1[:2],
            }
        )
    else:
        # setting the EOS or End of sentence to True
        features["EOS"] = True

    return features

In [ ]:
def sent2features(sent):
    """Convert sentences which are lists containing (w, p, t) into features"""
    return [word2features(sent, i) for i in range(len(sent))]

In [ ]:
def sent2labels(sent):
    """Retrieve all the labels from sentences which are lists containing (w, p, t)"""
    return [label for token, postag, label in sent]

In [ ]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [ ]:
# split into train and test
boundary = math.ceil(len(X) * 0.8)

In [ ]:
boundary

In [ ]:
# train data
x_train = X[:boundary]
y_train = y[:boundary]

# test data
x_test = X[boundary:]
y_test = y[boundary:]

In [ ]:
print(len(x_train))
print(len(x_test))

In [ ]:
# creating CRF model with Gradient Descent
crf = CRF(algorithm="lbfgs", c1=0.1, c2=0.1)

In [ ]:
# fitting the model using train data
crf.fit(x_train, y_train)

In [ ]:
# to get all the labels/tags of data
labels = list(crf.classes_)

In [ ]:
labels

Since we are not interested in 'O' tags we will check the performance of the CRF model using f1 scores for every tag except O tags.

In [ ]:
labels.remove("O")
labels

In [ ]:
# performing predictions based on the fitted model
y_pred = crf.predict(x_test)

In [ ]:
# finding the f1 score
metrics.flat_f1_score(y_test, y_pred, average="weighted", labels=labels)

In [ ]:
# finding the f1 score
print(metrics.flat_classification_report(y_test, y_pred, labels=labels, digits=3))

The model overfits!!

In [ ]:
eli5.show_weights(crf, top=30)

**Regularization**

In [ ]:
# creating a CRF Hyperparameter tuned model
crf_reg = CRF(algorithm="lbfgs", c1=5, c2=0.1)

In [ ]:
# fitting the hyperparameters
crf_reg.fit(x_train, y_train)

In [ ]:
# predict using the best CRF model
y_pred = crf_reg.predict(x_test)

In [ ]:
# print the f1 evaluation metric
print(metrics.flat_classification_report(y_test, y_pred, labels=labels, digits=3))

In [ ]:
eli5.show_weights(crf_reg, top=30)

## Untagged Data Stuff

In [ ]:
untagged_test_data = pd.read_csv("./Untagged Test Data/untagged_test_data.csv")

In [ ]:
untagged_test_data.head()

In [ ]:
len(untagged_test_data)

In [ ]:
agg_func_test = lambda s: [
    (w, p) for w, p in zip(s["word"].values.tolist(), s["pos"].values.tolist())
]

In [ ]:
grouped_test = untagged_test_data.groupby(["recipe_name", "Step#"]).apply(agg_func_test)

In [ ]:
test_sentences = [s for s in grouped_test]

In [ ]:
len(test_sentences)

In [ ]:
X_untagged_test = [sent2features(s) for s in test_sentences]

In [ ]:
len(X_untagged_test)

In [ ]:
pred_test = crf_reg.predict(X_untagged_test)

In [ ]:
len(pred_test)

In [ ]:
count = 0
for l in pred_test:
    for tag in l:
        if tag == "U-Action":
            count += 1

In [ ]:
count

In [ ]:
len(list(untagged_test_data["word"].values))

In [ ]:
len(list(set(untagged_test_data["word"].values)))

Boostrap output to the untagged data df

In [ ]:
flat_list = []
for sublist in pred_test:
    for item in sublist:
        flat_list.append(item)

In [ ]:
len(flat_list)

In [ ]:
untagged_test_data["Predicted Output"] = flat_list

In [ ]:
len(untagged_test_data[untagged_test_data["Predicted Output"] == "U-Action"])

In [ ]:
untagged_test_data[untagged_test_data["Predicted Output"] == "U-Action"]